# Master TSI Aix-Marseille Université: SIBIOM. UE Modalités émergentes/ TSI en IRM

Disclosure: this Jupyter is heavily borrowed from online materials found:

https://github.com/mikgroup/sigpy-mri-tutorial


Thanks to all scientists sharing their work !

Welcome !

In this notebook, we will learn to reconstruct MR-Images from k-space with various undersampling:
    a/ Partial Fourier
    b/ Parallel imaging
    c/ Compressed Sensing
    
The notebook uses SigPy to perform parallel imaging compressed sensing reconstruction. These iterative methods can be difficult to implement from scratch and computationally intensive. We will show how SigPy `App`s provide simple interfaces to run these reconstructions.

For a complete list of MRI `App`s, please see: https://sigpy.readthedocs.io/en/latest/mri_app.html

# Setup

MRI `App`s are separately defined in the mri submodule `sigpy.mri`. So in addition to Numpy and SigPy, we will need to import `sigpy.mri`.

In [ ]:
%matplotlib inline
import numpy as np
import sigpy as sp
import sigpy.mri as mr
import sigpy.plot as pl
import pygrappa

import matplotlib.pyplot as plt

For this notebook, we will be use an 8-channel brain dataset acquired with a Cartesian sampling. The dataset was retrospectively undersampled by 8 with a Poisson-disk undersampling mask. Let us load the dataset and visualize it:

In [ ]:
ksp = np.load('data/brain_ksp.npy')

print('kspace size is: ', ksp.shape, '\n')

pl.ImagePlot(ksp, mode='l', z=0, title='Log magnitude of k-space')

Please define the root sum of square operator as "rss"

Now let's look at the image

In [ ]:
 #img_rss = np.sum(np.abs(sp.ifft(ksp, axes=(-1, -2)))**2, axis=0)**0.5
img_rss = rss(sp.ifft(ksp, axes=(-1, -2)))

pl.ImagePlot(img_rss, title='Root-sum-of-squares image fully sampled')

# Adding noise to the k-space

Add a typical MRI-noise to the k-space. !! Recall MRI data are complex-valued  !!*

Note: you can skip this step first, go to the end of the notebook and come back to add noise and redo it all

In [ ]:
# define noise characteristics
mu, sigma = 0, 15e10# mean and standard deviation

# create a matrix of noise

# add the complex noise to kspace

# look at the noisy image :
pl.ImagePlot(rss(sp.ifft(noisyksp,axes=(-1,-2))), title='Magnitude of noisy image')

# Partial Fourier

 Let's try Partial Fourier undersampling & reconstruction using projection onto convex spaces (POCS) seen in class

In [ ]:
# Define the sampling mask (0 for missing data, 1 for sampled data)
mask = np.ones_like(ksp)
kspsz = ksp.shape

mask[:, np.arange(int(5/8*kspsz[1]),kspsz[1])] = 0  # Example: Simulate partial Fourier sampling

# Define the k-space data with missing values
usksp = ksp * mask

# look at the kspace
pl.ImagePlot(rss(usksp), mode='l', title='Log magnitude of undersampled k-space')

# look at the image now:
pl.ImagePlot(rss(sp.ifft(usksp,axes=(-1,-2))), title='Magnitude of partial Fourier image')

Define the center of kspace for calibration of parallel imaging 

In [ ]:
maskflip = np.flip(mask,axis=1)

pl.ImagePlot(maskflip*mask)

# POCS

Define a projection onto convex space (POCS) iterative algorithm that includes these steps:

    a/ Computes and save the image phase from symmetrically sampled low-frequency
    b/ Initiate the reconstruction with a 1st image (zero-filled) 
And iteratively:

    a/ Fourier transform estimated image to k-space 
    b/ Restore sampled data (fidelity)
    c/ Estimate the magnitude image
    d/ Combine the magnitude image with the phase image

In [ ]:

# Parameters for POCS reconstruction
num_iterations = 100


# Function to perform iterative POCS reconstruction
def iterative_pocs(kspace, mask, num_iterations=100 ):
   
    ## WRITE THE ALGORITHM HERE
    
    return recon


# Perform iterative POCS reconstruction
recon_result = iterative_pocs(usksp, mask)

In [ ]:
# look at it
pl.ImagePlot(rss(sp.ifft(usksp,axes=(-1,-2))), title='Magnitude of partial Fourier image')

# look at it
pl.ImagePlot(rss(recon_result), title='Magnitude of POCS-recon partial Fourier image')

Clearly the zero-filled reconstruction is not good! POCS is noisier but with sharper details

## Parallel imaging

In the following, we will perform parallel imaging compressed sensing reconstruction using Sigpy. We will first use ESPIRiT to estimate the sensitivity maps from the calibration region, then perform SENSE reconstruction, GRAPPA, L1 wavelet regularized reconstruction, and total variation regularized reconstruction.

In [ ]:
# let's skip one line out of 3 but keep the center of kspace (20 lines)


usksp = mask*ksp

print(usksp.shape)

pl.ImagePlot(usksp, mode='l', z=0, title='Log magnitude of uundersampled k-space')

# look at the image
pl.ImagePlot(rss(sp.ifft(usksp,axes=(-1,-2))), title='Magnitude of partial Fourier image')

To perform parallel imaging reconstruction, we will use the ESPIRiT method to estimate the sensitivity maps. To do this, we can use the [EspiritCalib](https://sigpy.readthedocs.io/en/latest/generated/sigpy.mri.app.EspiritCalib.html#sigpy.mri.app.EspiritCalib) App. 

To run an App, you simply do `app.run()`. You should be able to see a progress bar showing the `App`'s progress. 

Note that all MRI `App`s can run on GPU by specifying the option `device`. And of course, you will first need to have a GPU and install `cupy`.

In [ ]:
mps = mr.app.EspiritCalib(usksp).run()

pl.ImagePlot(mps, z=0, title='Sensitivity Maps Estimated by ESPIRiT')

## SENSE Recon

With the sensitivity maps ready, we can now run a SENSE reconstruction using the [SenseRecon](https://sigpy.readthedocs.io/en/latest/generated/sigpy.mri.app.SenseRecon.html#sigpy.mri.app.SenseRecon) `App`. We will use an l2 regularization of 0.01 based on trial-and-error with this dataset. Feel free to change it!

Again, we can run the reconstruction on GPU if we specify the `device` option.

In [ ]:
lamda = 0.001
img_sense = mr.app.SenseRecon(usksp, mps, lamda=lamda).run()

pl.ImagePlot(img_sense, title='SENSE Reconstruction')

## GRAPPA Recon

The parallel imaging reconstruction can be also performed in k-space using GRAPPA [Pygrappa](https://github.com/mckib2/pygrappa/tree/main/pygrappa).

In [ ]:
ncoils, sx, sy  = usksp.shape[:] 
print(sx,' ',sy,' ', ncoils)

ctrx,ctry, pd = int(sx/2), int(sy/2), 10 # center 20 lines are ACS
calib = usksp[:,ctrx-2*pd:ctrx+2*pd,ctry-pd:ctry+pd].copy() # call copy()!

# coil_axis=0 needs to be specified 
# alternatives are : grappa, cgrappa, igrappa, vcgrappa
res = pygrappa.cgrappa(np.cdouble(usksp), np.cdouble(calib), kernel_size=(4,4), coil_axis=0)
# res = pygrappa.igrappa(usksp, calib, kernel_size=(5,5), coil_axis=0)

# look at the kspace & image
pl.ImagePlot(rss(res),mode='l', title='Magnitude of GRAPPA kspace')
pl.ImagePlot(rss(sp.ifft(res,axes=(-1,-2))), title='Magnitude of GRAPPA image')

## L1 Wavelet Regularized Reconstruction

Similarly, we can perform an l1-wavelet regularized reconstruction using [L1WaveletRecon](https://sigpy.readthedocs.io/en/latest/generated/sigpy.mri.app.L1WaveletRecon.html#sigpy.mri.app.L1WaveletRecon).

In [ ]:
lamda = 0.005
img_l1wav = mr.app.L1WaveletRecon(usksp, mps, lamda).run()

pl.ImagePlot(img_l1wav, title='L1 Wavelet Regularized Reconstruction')



# Total Variation Recon

And we can do total variation regularized reconstruction with [TotalVariationRecon](https://sigpy.readthedocs.io/en/latest/generated/sigpy.mri.app.TotalVariationRecon.html#sigpy.mri.app.TotalVariationRecon).

In [ ]:
lamda = 0.05
img_tv = mr.app.TotalVariationRecon(usksp, mps, lamda).run()

pl.ImagePlot(img_tv, title='Total Variation Regularized Reconstruction')

And this is how you can do parallel imaging compressed sensing reconstruction with SigPy! We haven't gone through non-Cartesian datasets, but non-Cartesian support can be enabled by passing k-space coordinates to the `coord` option.
